# Item-Item Top-N Recommendations

In this excercise we will implement a simple top-N recommender, evaluate the algorithms, and then call algorithms from the Surprise package. In top-N recommendations the algorithm is requested to produce a list of N items that the user will be interested in. 
In this particular execercise we will work with an escape room dataset.

First, let's load the dataset, which is already split by time into a training set and a test set:

In [6]:
import pandas as pd
train_set_path = 'resources//train_numerized_with_anon.csv'
test_set_path = 'resources//test_numerized_with_anon.csv'

train_set = pd.read_csv(train_set_path, parse_dates=[3], index_col='index')
test_set = pd.read_csv(test_set_path, parse_dates=[3], index_col='index')

users_in_train = train_set.userID.unique()
test_set = test_set[test_set.userID.isin(users_in_train)]

We can take a look at the structure of the dataset:

In [7]:
train_set.head()

,userID,itemID,rating,timestamp
index,,,,
0,0,0,10,11/11/2015
1,1,0,10,11/11/2015
2,2,0,8,11/11/2015
3,3,0,10,11/11/2015
4,4,0,10,11/11/2015


## Part 1: Recommend Most Popular Items 

Now we can begin implementing the our first algorithm, that recommends to the user the list of most popular items. Although this is not a personalized approach, in many cases, this is not a bad idea - popular items are popular because everybody choose them, so there is a high likelihood that recommended popular items will be indeed chosen by the user.

In the code below, fill in the missing parts. The algorithm has a training method, where item popularity is computed, and a recommendation method, where the list of popular items.

In [8]:
import pandas as pd
import os

import_path = 'resources'

train_set_path = os.path.join(import_path, 'train_numerized_with_anon.csv')
test_set_path = os.path.join(import_path, 'test_numerized_with_anon.csv')

train_set = pd.read_csv(train_set_path, parse_dates=[3], index_col='index')
test_set = pd.read_csv(test_set_path, parse_dates=[3], index_col='index')

users_in_train = train_set.userID.unique()
test_set = test_set[test_set.userID.isin(users_in_train)]


class MostPopular:

    def __init__(self):
        self.item_ratings_sorted = None
        self.train_set = None

    def learn_model(self, train_set):

        # 1) Add code to set the item_ratings_sorted to the list of items in the training set,
        # ordered by decreasing popularity (i.e., the number of users who have chosen an item)

        self.train_set = train_set

        self.item_ratings_sorted = train_set.groupby(['itemID'])\
            .agg({'userID': 'nunique'}).sort_values(by='userID', ascending=False).index.to_list()

    def get_top_n_recommendations(self, test_set, top_n):
        result = {}

        already_ranked_items_by_user = self.train_set.groupby('userID')['itemID'].apply(list)

        # For each user in the test set compute recommendations

        # 2) Add to the result the top N items from the popular list
        # 3) Remove items that the user has already chosen in the training set (already_ranked_items_by_user)

        for userID in test_set.userID.unique():
            result[str(userID)] = [i for i in self.item_ratings_sorted
                                   if i not in already_ranked_items_by_user.loc[userID]][0:top_n]


        return result

    def clone(self):
        pass

Now we can call the most popular algorithm to deliver a list of reocmmendations. The code below prints the list of top 5 recommended items for user with ID 431.

In [9]:
popular = MostPopular()
popular.learn_model(train_set)
popular_recs = popular.get_top_n_recommendations(test_set,top_n=5)
print(popular_recs['431'])
assert popular_recs['431'] == [53, 26, 68, 85, 16], 'Wrong computation of popular items'

[53, 26, 68, 85, 16]


## Part 2 - Item-Item Recommendations

We now learn a slightly more sophisticated model, that uses item-item similarities. Given such a similarity score, we can recommend to a user items that are most similar to the items that the user has chosen in the past. One such useful similarity metric is the Jaccard coefficient. For two items i1 and i2, the Jaccard similarity is the number of users who have chosen both i1 and i2, divided by the number of users who have chosen either i1 or i2. That is, given the list of users who have chosen i1 and the list of users who have chosen i2, the Jaccard similarity is the intersection of the lists, divided by the union of the lists.

In practice, to expedite the recommendation process, and hence reduce online latency, we will compute the item-item co-occurence matrix in the model learning phase. Then, online, when recommendations are requested, we only need to compute for each item that the user has already chosen in the past, the Jaccard scores for the other items.

As the user has chosen several items in the past, we need to aggregate the Jaccard scores. That is, if the user has previously chosen i1 and i2, item i3 has two scores J(i1,i3) and J(i2,i3), and an aggregation of the scores is needed. There are two popular aggregation functions - sum and max. Empirically, max typically has better perfromance.

Fill in the missing parts in the code below.

In [10]:
from tqdm import tqdm
from itertools import combinations
from collections import defaultdict

import pandas as pd
import os

train_set_path = 'resources//train_numerized_with_anon.csv'
test_set_path = 'resources//test_numerized_with_anon.csv'

train_set = pd.read_csv(train_set_path, parse_dates=[3], index_col='index')
test_set = pd.read_csv(test_set_path, parse_dates=[3], index_col='index')

users_in_train = train_set.userID.unique()
test_set = test_set[test_set.userID.isin(users_in_train)]


class Jaccard:

    def __init__(self):
        self.item_ratings_sorted = None
        self.train_set = None
        self.item_item_counts = defaultdict(int)
        self.item_counts = None

    def learn_model(self, train_set):
        print('Started training')
        self.train_set = train_set
        self.item_counts = self.train_set.groupby('itemID')['userID'].agg('count').sort_values(ascending=False)

        maxpair = 0
        pbar = tqdm(total=len(train_set.userID.unique()))

        # Iterating over all item pairs is inefficient. It is better to iterate only over pairs of items that were chosen together.
        # Instead, we will iterate over the users, and for each user, and each two items that the user has chosen, increase the count

        for u in train_set.userID.unique():
            pbar.update(1)
            userData = self.train_set[self.train_set.userID == u]
            user_items_comb = list(combinations(userData['itemID'].unique(), 2))
            for pair_item in user_items_comb:
                self.item_item_counts[frozenset(pair_item)] += 1

            # 4) For each pair of items in the user data - increase the counts in self.item_item_counts

        pbar.close()
        print('Done training')

    def get_top_n_recommendations(self, test_set, top_n):
        print('Started computing recommendations')
        result = {}
        already_ranked_items_by_users = self.train_set.groupby('userID')['itemID'].apply(list)

        pbar = tqdm(total=len(test_set.userID.unique()))

        # For each user in the test set compute recommendations
        for userID in test_set.userID.unique():
            pbar.update(1)
            result[str(userID)] = []
            potential_items = [i for i in self.item_counts.index
                               if i not in already_ranked_items_by_users[userID]]
            # maxvalues will maintain for each potential item to recommend its highest Jaccard score.
            maxvalues = dict()

            # 4) Iterate over the items that the user has already chosen.
            # For each such item compute its Jaccard correlation to other items based on the item_item_counts.
            # Aggregate into maxvalues using max.
            # Avoid recommending items that the user has already chosen in the training set.
            for item in potential_items:
                max_value = 0
                for i in already_ranked_items_by_users[userID]:
                    intersection = self.item_item_counts[frozenset((i, item))]
                    union = self.item_counts[i] + self.item_counts[item]
                    score = intersection / union
                    if score > max_value:
                        max_value = score
                        maxvalues[item] = score

            # Now we just take the top N items by decreasing Jaccard
            top_list = sorted(maxvalues.items(), key=lambda kv: -kv[1])
            i = 0
            j = 0
            while i < top_n and j < len(top_list):
                itemID = top_list[j][0]
                j += 1
                result[str(userID)].append(itemID)
                i += 1

        pbar.close()
        print('Done computing recommendations')
        return result

    def clone(self):
        pass


In [11]:
jaccard = Jaccard()
jaccard.learn_model(train_set)
jaccard_recs = jaccard.get_top_n_recommendations(test_set,top_n=5)

  1%|          | 136/20197 [00:00<00:14, 1354.79it/s]

Started training


100%|██████████| 20197/20197 [00:13<00:00, 1489.02it/s]


Done training
Started computing recommendations


100%|██████████| 227/227 [00:12<00:00, 18.57it/s]

Done computing recommendations


As a side note - as computing the ite-item counts takes a while (especially with Python), we are using here the progress bar from the tqdm package (https://pypi.org/project/tqdm/). You need to install tqdm, or remove the progress bar, which would of course is not needed for the algorithm to run.

## Part 3 - Comparing the Algorithms

We now want to compare the recommendation lists to see which one is better. In top-N recommendations it is popular to computer the Precision@N metric - the portion of recommended items that were chosen by users in the test set. This is typically a reasonable metric for real systems, where one wants to optimize the number of recommended items that are chosen.

We compute Precision@N by comparing the number of recommendations chosen by the users, divided by the number of overall recommendations.

Fill in the missing parts in the code below:

In [12]:
def compute_precision(test_set, recommendations):
    #hits is the number of items that were recommended and chosen
    hits = 0
    #recs is the total number of recommended items
    recs = 0
    
    for u in test_set.userID.unique():
        userData = set(test_set[test_set.userID == u]['itemID'].to_list())
        userRecs = set(recommendations.get(str(u)))
        hits+=len(userRecs.intersection(userData))
        recs+=len(userRecs)
        #5) Compute here the number of hits. Update hits and recs accordingly.
        
    return hits / float(recs)

In [13]:
p1 = compute_precision(test_set, jaccard_recs)
p2 = compute_precision(test_set, popular_recs)
print("Jaccard=", p1, "  Popularity=", p2)

Jaccard= 0.03612334801762115   Popularity= 0.027312775330396475
